### Script to connect local spark to google cloud

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [9]:
credentials_location = '/Users/gdk/google-cloud-sdk/dtc-de-2023.json'

In [10]:
gcs_connector = '/Users/gdk/Projects/de-zoomcamp/spark/packages/gcs-connector-hadoop2-2.1.1.jar'

In [11]:
conf = SparkConf() \
    .setMaster('local[*]') \ # or the SPARK URL if you start standalone cluster locally
    .setAppName('test') \
    .set("spark.jars", gcs_connector) \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [14]:
sc = SparkContext.getOrCreate(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [15]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [16]:
# read data from google cloud storage
df_green = spark.read.parquet('gs://de-week4-data-lake/pq/green/*/*')

In [17]:
df_green.show(4)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-10 17:17:10|  2020-01-10 17:25:00|                 N|         1|          41|          74|              1|         1.12|        7.0|  1.0|    0.